# Transcript from lecture, October 12, 2021


In [1]:
import sys

########################################
# Change the string in the line below! #
########################################
sys.path.append("/Users/gilbert/Documents/CS111-2021-fall/Python") 

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
from scipy import integrate
import networkx as nx
import cs111

##########################################################
# If this import for matplotlib doesn't work, try saying #
#   conda install -c conda-forge ipympl                  #
# at a shell prompt on your computer                     #
##########################################################
import matplotlib
%matplotlib ipympl

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d




np.set_printoptions(precision = 4)

# How big is a vector?

## 2-norm of a vector (usually what we mean by just "norm")

In [2]:
v = np.array([3,1,4,1])
v

array([3, 1, 4, 1])

In [3]:
npla.norm(v)

5.196152422706632

In [4]:
v = np.random.random(10)
v

array([0.5411, 0.7526, 0.9859, 0.1827, 0.7715, 0.9704, 0.1039, 0.9323,
       0.7463, 0.8007])

In [5]:
npla.norm(v)

2.3408319377834106

In [6]:
npla.norm(2 * v)

4.681663875566821

## Error and residual for Ax=b, relative residual norm

In [7]:
A = np.random.random((5,5))
A

array([[0.3994, 0.3346, 0.1155, 0.8484, 0.6816],
       [0.2762, 0.0562, 0.3655, 0.0522, 0.6289],
       [0.4   , 0.8149, 0.2609, 0.1595, 0.2808],
       [0.3362, 0.4328, 0.4422, 0.6736, 0.1788],
       [0.7308, 0.1013, 0.8028, 0.3382, 0.0265]])

In [8]:
# make a b for which the exact answer is xhat = all ones
n = A.shape[0]
x_exact = np.ones(n)
print("x_exact:", x_exact)
print()

b = A @ x_exact
print("b:", b)

x_exact: [1. 1. 1. 1. 1.]

b: [2.3794 1.3789 1.9161 2.0635 1.9996]


In [9]:
x, r = cs111.LUsolve(A,b)
print("x:", x)

x: [1. 1. 1. 1. 1.]


In [10]:
error = x - x_exact
print("error:", error)

error: [-1.1102e-16  2.2204e-16  4.4409e-16 -2.2204e-16  0.0000e+00]


In [11]:
npla.norm(error)

5.551115123125783e-16

In [12]:
# If we don't know the exact answer, we don't know the error!

b = np.random.random(n)
print("b:", b)
print()

x, rel_res = cs111.LUsolve(A,b)
print("x:", x)

b: [0.6966 0.0738 0.3057 0.7367 0.2947]

x: [-0.4797  0.3139  0.3971  0.8715 -0.0031]


In [13]:
# But we can always compute the residual

residual = b - A@x
print("residual:", residual)
print()

print("residual norm:", npla.norm(b - A@x))
print()

print("relative residual norm:", npla.norm(b - A@x) / npla.norm(b))
print()


residual: [ 0.0000e+00  1.3878e-17  0.0000e+00 -1.1102e-16  1.1102e-16]

residual norm: 1.576213700060856e-16

relative residual norm: 1.430762471332042e-16



In [14]:
rel_res

1.430762471332042e-16

In [15]:
npla.norm(b)

1.101659941215399

In [16]:
# relative residual norm is immune to scaling.
# suppose the coefficients of the equations are in light-years, and we want to change to kilometers

km_per_lightyear = 9.461 * 10**12
AA = km_per_lightyear * A
bb = km_per_lightyear * b



In [17]:
print('A:\n', A)
print()
print('AA:\n', AA)

A:
 [[0.3994 0.3346 0.1155 0.8484 0.6816]
 [0.2762 0.0562 0.3655 0.0522 0.6289]
 [0.4    0.8149 0.2609 0.1595 0.2808]
 [0.3362 0.4328 0.4422 0.6736 0.1788]
 [0.7308 0.1013 0.8028 0.3382 0.0265]]

AA:
 [[3.7791e+12 3.1652e+12 1.0926e+12 8.0266e+12 6.4482e+12]
 [2.6129e+12 5.3146e+11 3.4576e+12 4.9356e+11 5.9504e+12]
 [3.7842e+12 7.7102e+12 2.4687e+12 1.5086e+12 2.6563e+12]
 [3.1803e+12 4.0945e+12 4.1832e+12 6.3728e+12 1.6915e+12]
 [6.9146e+12 9.5818e+11 7.5953e+12 3.1994e+12 2.5105e+11]]


In [18]:
xx, rr = cs111.LUsolve(AA, bb)
print("x:", x)
print()
print("xx:", xx)

x: [-0.4797  0.3139  0.3971  0.8715 -0.0031]

xx: [-0.4797  0.3139  0.3971  0.8715 -0.0031]


In [19]:
# residual depends on scaling, but relative residual norm does not

print("old residual:", b - A@x)
print("new residual:", bb - AA@xx)
print()

print("old residual norm:", npla.norm(b - A@x))
print("new residual norm:", npla.norm(bb - AA@xx))
print()

print("old relative residual norm:", npla.norm(b - A@x) / npla.norm(b))
print("new relative residual norm:", npla.norm(bb - AA@xx) / npla.norm(bb))
print()


old residual: [ 0.0000e+00  1.3878e-17  0.0000e+00 -1.1102e-16  1.1102e-16]
new residual: [ 0.001   0.0002  0.0005 -0.001   0.    ]

old residual norm: 1.576213700060856e-16
new residual norm: 0.0014850494458735887

old relative residual norm: 1.430762471332042e-16
new relative residual norm: 1.424807897750037e-16



In [21]:
npla.norm(xx   - x)

3.346937550298193e-16

# Other vector norms

In [22]:
npla.norm?

In [ ]:
v = np.array([3, 1, 4, 0, 5])
npla.norm(v)

In [23]:
print("2-norm:", npla.norm(v, 2))
print()
print("1-norm:", npla.norm(v, 1))
print()
print("inf-norm:", npla.norm(v, np.inf))
print()
print("10-norm:", npla.norm(v, 10))
print()
print("0-norm:", npla.norm(v, 0))   # not an actual norm! number of nonzeros

2-norm: 2.3408319377834106

1-norm: 6.787366478161063

inf-norm: 0.9858890616756333

10-norm: 1.0915513934025096

0-norm: 10.0


# How big is a matrix?   Rank, norm, condition number

## Rank of a matrix

In [24]:
A = np.array([[1, 4, 7], [2, 5, 8], [3, 6, 9]])
A

array([[1, 4, 7],
       [2, 5, 8],
       [3, 6, 9]])

In [25]:
A[:,1]

array([4, 5, 6])

In [26]:
(A[:,0] + A[:,2]) / 2

array([4., 5., 6.])

In [27]:
# A has a null vector v:
v = np.array([-1/2, 1, -1/2])
v

array([-0.5,  1. , -0.5])

In [28]:
A @ v

array([0., 0., 0.])

In [29]:
npla.matrix_rank(A)

2

<b> A square matrix is *singular* if its rank is smaller than its dimension,
    or equivalently if it has a nonzero null vector.

In [30]:
# LU factorization with partial pivoting doesn't work if the matrix is singular
L, U, p = cs111.LUfactor(A)

AssertionError: can't find nonzero pivot, matrix is singular

In [31]:
A.T

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [32]:
# The number of linearly independent rows is always 
# equal to the number of linearly independent columns.
# That is, the rank of A and A.T are the same

npla.matrix_rank(A.T)

2

In [33]:
# most (square) matrices have rank equal to their dimension
A = np.random.random((3,3))
A

array([[0.9491, 0.3745, 0.6807],
       [0.8056, 0.8443, 0.5151],
       [0.3053, 0.9718, 0.9562]])

In [34]:
npla.matrix_rank(A)

3

In [ ]:
A = A = np.array([[1, 2, 3], [2, 4, 6], [3, 6, 9]])
A

In [ ]:
npla.matrix_rank(A)

In [ ]:
# The rank of a non-square matrix can't be more than its smaller dimension
A = np.random.random((4,7))
A

In [ ]:
npla.matrix_rank(A)

In [ ]:
A.T

In [ ]:
npla.matrix_rank(A.T)

## Norm of a matrix

The norm of A, written ||A||, is the maximum over nonzero vectors v of norm(A@v) / norm(v)

Each different vector norm (1-norm, 2-norm, inf-norm, etc.) gives a different matrix norm.

In [35]:
A = np.array([[2,-1,1],[1,0,1],[3,-1,4]])
A

array([[ 2, -1,  1],
       [ 1,  0,  1],
       [ 3, -1,  4]])

In [36]:
npla.norm(A, 2)

5.722926953325028

In [37]:
npla.norm(A, 1)

6.0

<b> The 1-norm of a matrix turns out to be the largest column sum (summing absolute values)

In [38]:
npla.norm(A, np.inf)

8.0

<b> The 1-norm of a matrix turns out to be the largest row sum (summing absolute values)

## Condition number of a matrix

The condition number of A is the ratio (max norm(A@v)/norm(v)) / (min norm(A@v)/norm(v)), where both the max and the min are taken over nonzero vectors v.

The condition number is always greater than or equal to 1. If the rank of A is less than the number of columns, the min will be 0 and the condition number is infinite.

If A is square and nonsingular, the condition number is equal to ||A|| times ||A inverse||.

There is a different condition number for each norm: 1-norm condition, 2-norm condition, etc.

In [39]:
A = np.eye(5)
A

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [40]:
npla.cond(A, 2)

1.0

In [41]:
A = np.random.random((5,5))
A

array([[0.6857, 0.9609, 0.3269, 0.1615, 0.149 ],
       [0.7535, 0.7014, 0.7796, 0.7933, 0.5954],
       [0.1365, 0.943 , 0.5148, 0.4778, 0.3979],
       [0.9309, 0.2632, 0.5635, 0.7261, 0.0985],
       [0.0373, 0.3091, 0.0865, 0.8496, 0.2277]])

In [42]:
npla.cond(A, 2)

18.761166974899048

In [43]:
npla.cond(A, 1)

26.685106020276827

In [44]:
npla.cond(A, np.inf)

37.30801277559907

In [45]:
A = np.array([[1, 4, 7], [2, 5, 8], [3, 6, 9]])
A

array([[1, 4, 7],
       [2, 5, 8],
       [3, 6, 9]])

In [46]:
npla.cond(A,2)

LinAlgError: Singular matrix

In [ ]:
A = np.diag([1,2,3,4,5,6,7,8])
A

In [ ]:
npla.cond(A, 2)

## Scaling a matrix changes its norm but not its condition number

In [47]:
A = np.random.random((5,5))
A

array([[0.6323, 0.4218, 0.9103, 0.7347, 0.2241],
       [0.5199, 0.8875, 0.6751, 0.2991, 0.7338],
       [0.8401, 0.5605, 0.3784, 0.2434, 0.8697],
       [0.6218, 0.6665, 0.3788, 0.7167, 0.7681],
       [0.54  , 0.3243, 0.3577, 0.2469, 0.3437]])

In [48]:
npla.norm(A,2) 

2.83844588862938

In [49]:
npla.norm(10*A, 2)

28.3844588862938

In [50]:
npla.cond(A, 2)

51.81106999766571

In [51]:
npla.cond(10*A, 2)

51.81106999766577

## What about the determinant?

In [52]:
npla.det?

In [53]:
A = np.random.random((5,5))

In [54]:
npla.det(A)

-0.09742086868453018

In [55]:
npla.norm(A)

3.0760789190737587

In [56]:
npla.cond(A)

24.510020510341793

<b> The determinant is less useful than norm and condition number, especially as **n** grows.

In [57]:
n = 10
A = 2 * np.eye(n)
print('A:\n', A)

A:
 [[2. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 2.]]


In [58]:
print('n:', n, 'norm:', npla.norm(A,2), 'condition number:', npla.cond(A), 'determinant:', npla.det(A))

n: 10 norm: 2.0 condition number: 1.0 determinant: 1024.0


In [59]:
n = 100
A = 2 * np.eye(n)
print('n:', n, 'norm:', npla.norm(A,2), 'condition number:', npla.cond(A), 'determinant:', npla.det(A))

n: 100 norm: 2.0 condition number: 1.0 determinant: 1.2676506002283037e+30
